# Use `get_rm_from_skycoord` and compare with values from RMextract

This example script shows how to get ionospheric rotation measures for a give source,
observer location and array of times. This script uses the default ionospheric settings,
which are good for most purposes.

In [ ]:
from __future__ import annotations

import astropy.units as u
import matplotlib.pyplot as plt
import numpy as np
from astropy.coordinates import EarthLocation, SkyCoord
from astropy.time import Time
from astropy.visualization import quantity_support, time_support
from spinifex import get_rm

In [ ]:
### Required to load local data for example - not needed for normal use
from importlib import resources

with resources.as_file(resources.files("spinifex.data.tests")) as datapath:
    spinifex_data = datapath
###

In [ ]:
times = Time("2020-01-08T01:13:00") + np.arange(10) * 1 * u.hr
# create source object
source = SkyCoord(ra=350.85 * u.deg, dec=58.815 * u.deg)
# create Earth Location
lon = 6.367 * u.deg
lat = 52.833 * u.deg
dwingeloo = EarthLocation(lon=lon, lat=lat, height=0 * u.km)
# get rotation measures
rm = get_rm.get_rm_from_skycoord(
    loc=dwingeloo,
    times=times,
    source=source,
    # We set these options to use the data packaged with spinifex
    # Unsetting them will cause the function to download the data from the internet
    output_directory=spinifex_data,
)

To get the values from the same server, source and time from RMextract, install RMextract and uncomment the following lines:

In [ ]:
# import RMextract.getRM as gt
# starttime = times[0].mjd*24*3600.  # getRM still wants MJD time in seconds (casacore definition)
# endtime = times[-1].mjd*24*3600
# statpos = [dwingeloo.x.to(u.m).value, dwingeloo.y.to(u.m).value, dwingeloo.z.to(u.m).value]
# pointing=[ source.ra.rad,  source.dec.rad ] #3C196  Ra, Dec in radians
# RMdict = gt.getRM(ionexPath='./ionex_data/', radec=pointing, timestep=3600, timerange = [starttime, endtime],
# stat_positions=[statpos,],earth_rot=1, prefix='uqrg', server="http://chapman.upc.es")


Here we directly take the values we got from the above code, to avoid installing RMextract

In [ ]:
RMdict = {}
RMdict["RM"] = {}
RMdict["RM"]["st1"] = np.array(
    [
        [0.10242756],
        [0.08804738],
        [0.07332737],
        [0.08251355],
        [0.07816817],
        [0.08137425],
        [0.11095578],
        [0.15794444],
        [0.26828872],
        [0.36951645],
        [0.44763038],
        [0.49045776],
    ]
)
RMdict["times"] = np.array(
    [
        5.08515918e09,
        5.08516278e09,
        5.08516638e09,
        5.08516998e09,
        5.08517358e09,
        5.08517718e09,
        5.08518078e09,
        5.08518438e09,
        5.08518798e09,
        5.08519158e09,
        5.08519518e09,
        5.08519878e09,
    ]
)
RMdict["azimuth"] = {}
RMdict["azimuth"]["st1"] = np.array(
    [
        -0.5065002,
        -0.37364739,
        -0.23410709,
        -0.09012949,
        0.05561559,
        0.20026593,
        0.34108016,
        0.475697,
        0.60222255,
        0.7190818,
        0.82458167,
        0.9159993,
    ]
)
RMdict["elev"] = {}
RMdict["elev"]["st1"] = np.array(
    [
        0.52492472,
        0.45731976,
        0.40983145,
        0.38421338,
        0.38147449,
        0.40172543,
        0.44416086,
        0.50718446,
        0.58862976,
        0.68600301,
        0.79668206,
        0.91802653,
    ]
)

In [ ]:
rm_rme = RMdict["RM"]["st1"]
times_rme = Time(RMdict["times"] / (3600 * 24), format="mjd")
az_rme = np.degrees(RMdict["azimuth"]["st1"])
elev_rme = np.degrees(RMdict["elev"]["st1"])

In [ ]:
# print to screen
rotation_measures = rm.rm
rm_times = rm.times
elevations = rm.elevation
azimuths = rm.azimuth
print("time      RM (rad/lambda^2)      azimuth (deg)      elevation (deg)")
for myrm, tm, az, el in zip(rotation_measures, rm_times, azimuths, elevations):
    print(f"{tm.isot} {myrm} {az:3.2f} {el:2.2f}")
print("Values from RMextract:")
for myrm, tm, az, el in zip(rm_rme, times_rme, az_rme, elev_rme):
    print(f"{tm.isot} {myrm} {az:3.2f} {el:2.2f}")

In [ ]:
with time_support(), quantity_support():
    fig, ax = plt.subplots()
    im = ax.errorbar(rm.times.datetime, rm.rm, rm.rm_error, fmt=".", label="spinifex")
    im2 = ax.plot(times_rme.datetime, rm_rme, "o-", label="RMextract")
    ax.set_ylabel("RM / rad/m$^2$")
    ax.legend()
    plt.xticks(rotation=20)